In [6]:
import os
import pandas as pd
import spacy
import csv
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
import random
from spacy.util import minibatch, compounding
from spacy.training.example import Example


In [3]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

You should consider upgrading via the '/opt/conda/bin/python3.10 -m pip install --upgrade pip' command.
2023-06-10 22:05:01.287433: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 18.7 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
# data
from subprocess import check_output
print(check_output(["ls", "/home/jovyan/work/data/label_data"]).decode("utf8"))

digitoday.2014.csv
digitoday.2014.dev.csv
digitoday.2014.train.csv
digitoday.2015.test.csv
ner.csv



In [5]:
dframe = pd.read_csv("/home/jovyan/work/data/label_data/ner.csv")
dframe.head(30)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x85 in position 257491: invalid start byte

In [4]:
grouped = dframe.groupby("sentence_idx").apply(lambda s: [(w, t) for w, t in zip(s["word"].values.tolist(),s["tag"].values.tolist())])
sentences = [s for s in grouped]

In [5]:
test = [[('TokyoiTower', 'O'),
  ('of', 'O'),
  ('demonstrators', 'O'),
  ('have', 'O'),
  ('marched', 'O'),
  ('through', 'O'),
  ('London', 'B-geo'),
  ('to', 'O'),
  ('protest', 'O')],[
  ('the', 'O'),
  ('war', 'O'),
  ('in', 'O'),
  ('Iraq', 'B-geo')]]
data = []
for s in sentences:
    full_s = ""
    start_index = 0
    last_index = 0
    index_counter = 0
    entities = []
    for w in s:
        full_s += str(w[0]) + " "
        entities.append((index_counter, index_counter+len(str(w[0])),w[1]))
        index_counter += len(str(w[0])) + 1



    data.append((full_s,{"entities":entities}))

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


In [6]:
# Initialize a new blank spaCy model
nlp = spacy.load("en_core_web_sm")

# Create the NER component
# ner = nlp.add_pipe("ner", name="ner", last=True)
ner = nlp.get_pipe("ner")

In [7]:
# Add custom tags to the NER pipeline
for _, annotations in data:
    for entity in annotations.get("entities"):
        ner.add_label(entity[2])

# Disable unnecessary pipeline components
pipe_exceptions = ["ner"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]


In [8]:
# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):
    # Training iterations...
    for iteration in range(30):
        # Shuffle training examples before every iteration
        random.shuffle(train_data)
        losses = {}
        examples = []

        # Create Example objects from training data
        for text, annotation in train_data:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotation)
            examples.append(example)

        # Batch up the examples using spaCy's minibatch
        batches = minibatch(examples, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            nlp.update(batch, drop=0.5, losses=losses)

        print("Losses:", losses)


/opt/conda/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "In a telephone interview to discuss the issues at ..." with entities "[(0, 2, 'O'), (3, 4, 'O'), (5, 14, 'O'), (15, 24, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "" And I think they 'll want a one-stop shop in ter..." with entities "[(0, 1, 'O'), (2, 5, 'O'), (6, 7, 'O'), (8, 13, 'O...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses: {'ner': 73734.74145173073}
Losses: {'ner': 53878.20321031019}
Losses: {'ner': 49302.55247410178}
Losses: {'ner': 46414.35323068049}
Losses: {'ner': 44850.43970225345}
Losses: {'ner': 43325.47696243262}
Losses: {'ner': 41832.97139682325}
Losses: {'ner': 40887.51726655343}
Losses: {'ner': 39685.220488571635}
Losses: {'ner': 39052.11174935772}
Losses: {'ner': 38378.04163152038}
Losses: {'ner': 37839.33275748836}
Losses: {'ner': 37123.17211381531}
Losses: {'ner': 36667.88268107121}
Losses: {'ner': 36329.202935263434}
Losses: {'ner': 35749.99659053747}
Losses: {'ner': 35407.37053461587}
Losses: {'ner': 35237.78135659149}
Losses: {'ner': 34691.79147212618}
Losses: {'ner': 34490.66869269444}
Losses: {'ner': 34232.43216443671}
Losses: {'ner': 33843.881472823545}
Losses: {'ner': 33929.466078790465}
Losses: {'ner': 33448.6395753923}
Losses: {'ner': 33306.60424603189}
Losses: {'ner': 33154.80018921654}
Losses: {'ner': 32673.55635082893}
Losses: {'ner': 32892.41659821806}
Losses: {'ner': 3

In [14]:
# Save the trained model
output_dir = "/home/jovyan/work/data/output/spacyModel"
nlp.to_disk(output_dir)

In [5]:
# # Testing the model This is for the other parameter
# test_examples = []
# for text, annotation in test_data:
#     doc = nlp.make_doc(text)
#     example = Example.from_dict(doc, annotation)
#     test_examples.append(example)
#
# # Evaluate the model on the test set
# scores = nlp.evaluate(test_examples)
# print("Test Scores:", scores)

# This is for accuracy
test_examples = []
for text, annotation in test_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotation)
    test_examples.append(example)

# Evaluate the model on the test set
scores = nlp.evaluate(test_examples)

# Calculate accuracy for each tag
tag_accuracies = {}
for tag, values in scores['ents_per_type'].items():
    tag_accuracy = round(values['f'] * 100, 2)  # Convert F1 score to percentage and round to 2 decimal places
    tag_accuracies[tag] = tag_accuracy

# Print accuracy for each tag
print("Accuracy per Tag:")
for tag, accuracy in tag_accuracies.items():
    print("- {}: {}%".format(tag, accuracy))


NameError: name 'test_data' is not defined

In [17]:
token_accuracy = scores['token_acc']
accuracy = round(token_accuracy * 100, 2)  # Convert to percentage and round to 2 decimal places

print("Accuracy: {}%".format(accuracy))

Accuracy: 100.0%


In [16]:
entity_metrics = scores['ents_per_type']

precision_sum = 0.0
recall_sum = 0.0
f_score_sum = 0.0
num_entity_types = len(entity_metrics)

for entity_type, metrics in entity_metrics.items():
    precision_sum += metrics['p']
    recall_sum += metrics['r']
    f_score_sum += metrics['f']

general_precision = precision_sum / num_entity_types
general_recall = recall_sum / num_entity_types
general_f_score = f_score_sum / num_entity_types

print("General Precision:", general_precision)
print("General Recall:", general_recall)
print("General F-Score:", general_f_score)

General Precision: 0.7191595723387469
General Recall: 0.646973869330037
General F-Score: 0.669365490491828
